In [0]:
import pandas as pd
import numpy as np

In [0]:
df=pd.read_csv('/content/sample_data/datasets.csv')

In [3]:
df=df.fillna(0)
df

,SEMESTER,AGE,GENDER,ADDRESS,FAMILY SIZE,PARENT STATUS,TRAVEL TIME,FAMILY SUPPORT,ACTIVITIES,LANGUAGES,INTERNET,FREETIME,HEALTH,NO OF ABSENCE,NO OF WITHDRAWL,CURRENT CGPA,STUDY TIME,STUDENT JOB,AVG GPA
0,10,18,0,0,1,1,1,0,0,1,2,4,2,1,1.0,2,4,0,4.00
1,10,18,1,1,0,0,2,0,0,0,3,4,3,9,4.0,3,3,1,4.00
2,5,23,1,1,1,1,2,1,1,1,5,1,5,2,2.0,2,2,1,1.67
3,5,20,1,1,0,0,3,1,1,0,5,5,1,4,2.0,5,3,0,3.67
4,6,23,0,0,0,1,1,1,0,0,4,1,3,5,2.0,2,5,0,3.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30256,10,24,0,1,1,0,2,0,0,0,5,1,5,8,5.0,1,4,1,2.33
30257,1,21,0,0,0,1,4,1,1,0,4,5,5,9,1.0,4,4,0,2.00
30258,9,22,0,0,0,0,4,1,1,1,1,4,3,1,5.0,1,3,1,2.33
30259,5,21,0,0,1,0,2,0,0,0,5,1,4,4,5.0,3,2,1,2.33


In [4]:
x_sum=np.sum(df.isnull())
print(x_sum)

SEMESTER           0
AGE                0
GENDER             0
ADDRESS            0
FAMILY SIZE        0
PARENT STATUS      0
TRAVEL TIME        0
FAMILY SUPPORT     0
ACTIVITIES         0
LANGUAGES          0
INTERNET           0
FREETIME           0
HEALTH             0
NO OF ABSENCE      0
NO OF WITHDRAWL    0
CURRENT CGPA       0
STUDY TIME         0
STUDENT JOB        0
AVG GPA            0
dtype: int64


In [5]:
df.columns

Index(['SEMESTER', 'AGE', 'GENDER', 'ADDRESS', 'FAMILY SIZE', 'PARENT STATUS',
       'TRAVEL TIME', 'FAMILY SUPPORT', 'ACTIVITIES', 'LANGUAGES', 'INTERNET',
       'FREETIME', 'HEALTH', 'NO OF ABSENCE', 'NO OF WITHDRAWL',
       'CURRENT CGPA', 'STUDY TIME', 'STUDENT JOB', 'AVG GPA'],
      dtype='object')

In [0]:
x = df.drop(['AVG GPA'], axis=1)
y = df['AVG GPA']

In [7]:
x.shape,y.shape

((30261, 18), (30261,))

In [8]:
y = df['AVG GPA']>2.7
y

0         True
1         True
2        False
3         True
4         True
         ...  
30256    False
30257    False
30258    False
30259    False
30260    False
Name: AVG GPA, Length: 30261, dtype: bool

In [9]:
num_feats=8
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(x, y,num_feats)
print(str(len(cor_feature)), 'Selected Features')

8 Selected Features


In [10]:
cor_feature

['AGE',
 'GENDER',
 'STUDY TIME',
 'CURRENT CGPA',
 'SEMESTER',
 'FAMILY SUPPORT',
 'STUDENT JOB',
 'FAMILY SIZE']

In [11]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(x)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = x.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

8 selected features


In [12]:
chi_feature

['SEMESTER',
 'GENDER',
 'FAMILY SIZE',
 'FAMILY SUPPORT',
 'ACTIVITIES',
 'CURRENT CGPA',
 'STUDY TIME',
 'STUDENT JOB']

In [14]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)
rfe_support = rfe_selector.get_support()
rfe_feature = x.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

Fitting estimator with 18 features.
8 selected features


In [15]:
rfe_feature

['SEMESTER',
 'AGE',
 'FAMILY SIZE',
 'FAMILY SUPPORT',
 'INTERNET',
 'CURRENT CGPA',
 'STUDY TIME',
 'STUDENT JOB']

In [16]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = x.loc[:,embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')

6 selected features


In [17]:
embeded_lr_feature

['SEMESTER',
 'AGE',
 'FAMILY SIZE',
 'FAMILY SUPPORT',
 'CURRENT CGPA',
 'STUDENT JOB']

In [18]:

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(x, y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = x.loc[:,embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')

8 selected features


In [19]:
embeded_rf_feature

['SEMESTER',
 'AGE',
 'INTERNET',
 'FREETIME',
 'NO OF ABSENCE',
 'NO OF WITHDRAWL',
 'CURRENT CGPA',
 'STUDY TIME']

In [20]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=num_feats)
embeded_lgb_selector.fit(x, y)

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = x.loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

8 selected features


In [21]:
embeded_lgb_feature

['SEMESTER',
 'AGE',
 'FREETIME',
 'HEALTH',
 'NO OF ABSENCE',
 'NO OF WITHDRAWL',
 'CURRENT CGPA',
 'STUDY TIME']

In [0]:
feature_selection_df = pd.DataFrame({'Feature':x, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embeded_lr_support,
                                    'Random Forest':embeded_rf_support, 'LightGBM':embeded_lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,"(S, E, M, E, S, T, E, R)",True,True,True,True,True,True,6
2,"(A, G, E)",True,True,True,True,True,True,6
3,"(N, O, , O, F, , A, B, S, E, N, C, E)",True,True,True,False,True,True,5
4,"(C, U, R, R, E, N, T, , C, G, P, A)",True,True,True,True,False,False,4
5,"(T, R, A, V, E, L, , T, I, M, E)",True,True,True,False,False,False,3
6,"(N, O, , O, F, , W, I, T, H, D, R, A, W, L)",True,True,True,False,False,False,3
7,"(I, N, T, E, R, N, E, T)",True,True,True,False,False,False,3
8,"(H, E, A, L, T, H)",True,True,True,False,False,False,3


In [0]:
#Recursive Feature Elimination
x=df[['SEMESTER','AGE','ADDRESS','FAMILY SIZE','HEALTH','FAMILY SUPPORT','STUDY TIME','NO OF WITHDRAWL']]

In [0]:
#Chi-Squared
x=df[['SEMESTER','ADDRESS','FAMILY SIZE','HEALTH','FAMILY SUPPORT','STUDY TIME','NO OF WITHDRAWL']]

In [0]:
#Pearson Correlation
x=df[['SEMESTER','AGE','ADDRESS','FAMILY SIZE','FAMILY SUPPORT','STUDY TIME','NO OF WITHDRAWL']]

In [0]:
#lasso: SelectFromModel l2
x=df[['AGE','FAMILY SIZE','FAMILY SUPPORT','STUDY TIME','NO OF WITHDRAWL']]

In [0]:
#tree-based: SelectFromModel
x=df[['SEMESTER','AGE','INTERNET','FREETIME','HEALTH','NO OF ABSENCE','NO OF WITHDRAWL','CURRENT CGPA']]

In [0]:
#lightGBM
x=df[['SEMESTER','AGE','TRAVEL TIME','INTERNET','HEALTH','NO OF ABSENCE','NO OF WITHDRAWL','CURRENT CGPA']]